In [8]:
## please delete below code after convertion in converted script(py) file
## + 필요없는 내용 삭제(초반부 1,3~14열, In[ ]형태의 주석제거)
!jupyter nbconvert --to script augment_cnn.ipynb
!sed -i '/^#[ ]In\[/d' augment_cnn.py
!sed -i -e '1d;3,14d' augment_cnn.py

[NbConvertApp] Converting notebook augment_cnn.ipynb to script
[NbConvertApp] Writing 3237 bytes to augment_cnn.py


In [ ]:
import torch
import torch.nn as nn
from models.augment_cells import AugmentCell
from models import ops

In [2]:
class AuxiliaryHead(nn.Module):
    def __init__(self, input_size, C, n_classes):
        assert input_size in [7,8]
        super().__init__()
        self.net = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.AvgPool2d(5, stride=input_size-5, padding=0, count_include_pad=False), # 2x2 out
            nn.Conv2d(C, 128, kernel_size=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 768, kernel_size=2, bias=False), #1x1 out
            nn.BatchNorm2d(768),
            nn.ReLU(inplace=True)
        )
        self.linear = nn.Linear(768, n_classes)
        
    def forward(self, x):
        out = self.net(x)
        out = out.view(out.size(0), -1) #flatten
        logits = self.linear(out)
        return logits

In [5]:
class AugmentCNN(nn.Module):
    def __init__(self, input_size, C_in, C, n_classes, n_layers, auxiliary, genotype, stem_multiplier=3):
        super().__init__()
        self.C_in = C_in
        self.C = C
        self.n_classes = n_classes
        self.n_layers = n_layers
        self.genotype = genotype
        
        self.aux_pos = 2*n_layers//3 if auxiliary else -1
        
        C_cur = stem_multiplier * C
        self.stem = nn.Sequential(
            nn.Conv2d(C_in, C_cur, 3,1,1,bias=False),
            nn.BatchNorm2d(C_cur)
        )
    
        C_pp, C_p, C_cur = C_cur, C_cur, C
        
        self.cells = nn.ModuleList()
        reduction_p = False
        for i in range(n_layers):
            if i in [n_layers//3, 2*n_layers//3]:
                C_cur *= 2
                reduction = True
            else:
                reduction = False
                
            cell = AugmentCell(genotype, C_pp, C_p, C_cur, reduction_p, reduction)
            reduction_p = reduction
            self.cells.append(cell)
            C_cur_out = C_cur * len(cell.concat)
            C_pp, C_p = C_p, C_cur_out
            
            if i == self.aux_pos:
                self.aux_head = AuxiliaryHead(input_size//4, C_p, n_classes)
        
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.linear = nn.Linear(C_p, n_classes)
        
    def forward(self, x):
        s0 = s1 = self.stem(x)
        
        aux_logits = None
        for i, cell in enumerate(self.cells):
            s0, s1 = s1, cell(s0, s1)
            if i == self.aux_pos and self.training:
                aux_logits = self.aux_head(s1)
        
        out = self.gap(s1)
        out = out.view(out.size(0), -1)
        logits = self.linear(out)
        return logits, aux_logits
    
    def drop_path_prob(self, p):
        for module in self.modules():
            if isinstance(module, ops.DropPath_):
                module.p = p